# Synaptic plasticity

In [ ]:
#@title Run the following to initialize lab environment.

!pip install ipympl ipywidgets stg-net -q

import matplotlib.pyplot as plt         # import matplotlib
from matplotlib.widgets import Slider
import numpy as np                      # import numpy
import ipywidgets as widgets            # interactive display

# Colab setting for widget
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except ImportError:
    pass

# modeling library
from stg_net.neuron import LIF
from stg_net.input import Poisson_generator, Gaussian_generator, Current_injector
from stg_net.conn import Simulator
from stg_net.helper import plot_volt_trace

# setting for figures
fig_w, fig_h = 8, 6
my_fontsize = 18
my_params = {'axes.labelsize': my_fontsize,
          'axes.titlesize': my_fontsize,
          'figure.figsize': (fig_w, fig_h),
          'font.size': my_fontsize,
          'legend.fontsize': my_fontsize-4,
          'lines.markersize': 8.,
          'lines.linewidth': 2.,
          'xtick.labelsize': my_fontsize-2,
          'ytick.labelsize': my_fontsize-2}
plt.rcParams.update(my_params)
my_layout = widgets.Layout()

# Auto Reloading
%load_ext autoreload
%autoreload 2

# Widget interaction
%matplotlib widget

## Different Types of Synaptic Plasticity

Synaptic weights can change based on the activity patterns of pre-synaptic and post-synaptic neurons.

### Rate-Dependent Plasticity

Synaptic weight changes depend solely on the firing rates of the pre- and post-synaptic neurons.

### Spike-Timing-Dependent Plasticity (STDP)

Synaptic weight changes depend on the precise timing of spikes between the pre- and post-synaptic neurons.

In the following exercises, you will explore key properties of these two forms of synaptic plasticity.


In [ ]:
#@title Run the following to start rate-dependant plasticity simulation { vertical-output: true }
T, dt = 1e3, 0.1        # simulation period(ms), step size(ms)
N = 2                  # number of neurons

# neuron types
tonic_neuron = {'tau_m':20., 'a':0., 'tau_w':30., 'b':3., 'V_reset':-55.}
adapting_neuron = {'tau_m':20., 'a':0., 'tau_w':100., 'b':0.5, 'V_reset':-55.}
initburst_neuron = {'tau_m':10., 'a':0., 'tau_w':100., 'b':1., 'V_reset':-50.}
bursting_neuron = {'tau_m':5., 'a':0., 'tau_w':100., 'b':1., 'V_reset':-46.}
irregular_neuron = {'tau_m':10., 'a':-0.01, 'tau_w':50., 'b':1.2, 'V_reset':-46.}
transient_neuron = {'tau_m':5., 'a':0.05, 'tau_w':100., 'b':0.7, 'V_reset':-60.}
delayed_neuron = {'tau_m':5., 'a':-0.1, 'tau_w':100., 'b':1., 'V_reset':-60.}
rebound_neuron = {'tau_m':5., 'a':0.2, 'tau_w':150., 'b':0.1, 'V_reset':-54.}

neuron_params = {'tonic_neuron': tonic_neuron, 'adapting_neuron': adapting_neuron,
                 'initburst_neuron': initburst_neuron, 'bursting_neuron': bursting_neuron,
                 'irregular_neuron': irregular_neuron, 'transient_neuron': transient_neuron,
                 'delayed_neuron': delayed_neuron, 'rebound_neuron': rebound_neuron}

# input types
Itypes = ['Icur', 'Gaussian', 'Poisson']

# updating parameters
def update_rate_plasticity(J_ts=1.0, rt=50., rs=50., Itype='Icur'):
    # simualtor
    h = Simulator(dt=dt)

    # network of neurons
    nrns = [LIF(sim=h) for _ in range(N)]
    for nrn in nrns:
        nrn.update(tonic_neuron)

    # background noise
    if Itype == 'Icur':
        noises = [Current_injector(sim=h, rate=r) for r in [rt, rs]]
    elif Itype == 'Gaussian':
        noises = [Gaussian_generator(sim=h, mean=r, std=r) for r in [rt, rs]]
    elif Itype == 'Poisson':
        noises = [Poisson_generator(sim=h, rate=r*3) for r in [rt, rs]]
    else:
        print('Invalid input')
    for noise, nrn in zip(noises, nrns):
        nrn.connect(noise, {'ctype':'Static', 'weight':1e0, 'delay':5})

    # recurrent connections
    tps = [['Static' for _ in range(N)] for _ in range(N)]
    tps[0][1] = 'Hebb'
    con = np.array([[0., J_ts],
                    [0., 0.]])
    dly = np.random.uniform(2., 5., (N,N))
    synspecs = [[{} for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            synspecs[i][j] = {'ctype':tps[i][j], 'weight':con[i,j], 'delay':dly[i,j]}
    cons = h.connect(nrns, nrns, synspecs)

    # simulation
    h.run(T)

    # visualize
    plt.clf()
    cs = ['b', 'r']
    plt.subplot(1,2,1)
    plt.title('raster')
    for nrn, c, l in zip(nrns, cs, range(N)):
        plt.eventplot(nrn.spikes['times'], lineoffsets=2*l, colors=c, label='%.1fHz'%(len(nrn.spikes['times'])/T*1e3*2))
    plt.xlabel('Time(ms)')
    plt.yticks(list(np.arange(N)*2), ['T', 'S'])
    plt.xlim([0., T])
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(np.arange(0., T, dt), cons[0][1].weights, c='purple')
    for nrn, c in zip(nrns, cs):
        plt.eventplot(nrn.spikes['times'], lineoffsets=0, linelengths=0.5, colors=c)
    plt.xlabel('Time(ms)')
    plt.ylabel('Relative amp')
    plt.tight_layout()

try:
    plt.figure(fig_rate_plasticity)
    plt.clf()
except:
    print('Init figure')
fig_rate_plasticity, axes = plt.subplots(1,2,figsize=(10,5))
widgets.interact(update_rate_plasticity, J_ts=(0.01, 2., 0.01), rt=(30., 70., 1.), rs=(30., 70., 1.), Itype=Itypes);

## To-Do

Investigate how the final synaptic weight change depends on the firing rates of the pre- and post-synaptic neurons.  
Systematically vary the input rates to both neurons and measure the resulting weight change as a function of pre- and post-synaptic firing rates.

## Think

Suppose the pre- and post-synaptic neurons are firing at rates $Pre$ and $Post$, respectively.  
Assuming **rate-dependent Hebbian plasticity**, which of the following scenarios is likely to produce a larger synaptic weight change?

1. Both pre- and post-synaptic neurons fire regularly  
2. Both pre- and post-synaptic neurons fire irregularly (Poisson firing)  
3. Pre-synaptic neuron fires regularly; post-synaptic neuron fires irregularly (Poisson)  
4. Pre-synaptic neuron fires irregularly (Poisson); post-synaptic neuron fires regularly  

What might explain the differences in synaptic weight change across these conditions?


In [ ]:
#@title Run the following to start spike-dependent plasticity simulation { vertical-output: true }
def update_spike_plasticity(delay_s=1., delay_t=5., Itype='Icur'):
    # simualtor
    h = Simulator(dt=dt)

    # network of neurons
    nrns = [LIF(sim=h) for _ in range(N)]
    for nrn in nrns:
        nrn.update(tonic_neuron)

    # background noise
    rt = rs = 40.
    if Itype == 'Icur':
        noises = [Current_injector(sim=h, rate=r, start=int(T/dt*0.25), end=int(T/dt*0.75)) for r in [rt, rs]]
    elif Itype == 'Gaussian':
        noises = [Gaussian_generator(sim=h, mean=r, std=r, start=int(T/dt*0.25), end=int(T/dt*0.75)) for r in [rt, rs]]
    elif Itype == 'Poisson':
        noises = [Poisson_generator(sim=h, rate=r*3, start=int(T/dt*0.25), end=int(T/dt*0.75)) for r in [rt, rs]]
    else:
        print('Invalid input')
    for noise, nrn, delay in zip(noises, nrns, [delay_t, delay_s]):
        nrn.connect(noise, {'ctype':'Static', 'weight':1e0, 'delay':delay})

    # recurrent connections
    tps = [['Static' for _ in range(N)] for _ in range(N)]
    tps[0][1] = 'STDP'
    con = np.array([[0., 1.0],
                    [0., 0.]])
    dly = np.random.uniform(2., 5., (N,N))
    synspecs = [[{} for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            synspecs[i][j] = {'ctype':tps[i][j], 'weight':con[i,j], 'delay':dly[i,j]}
    cons = h.connect(nrns, nrns, synspecs)

    # simulation
    h.run(T)

    # visualize
    plt.clf()
    cs = ['b', 'r']
    plt.subplot(1,2,1)
    plt.title('raster')
    for nrn, c, l in zip(nrns, cs, range(N)):
        plt.eventplot(nrn.spikes['times'], lineoffsets=2*l, colors=c, label='%.1fHz'%(len(nrn.spikes['times'])/T*1e3*2))
    plt.xlabel('Time(ms)')
    plt.yticks(list(np.arange(N)*2), ['T', 'S'])
    plt.xlim([0., T])
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(np.arange(0., T, dt), cons[0][1].weights, c='purple')
    for nrn, c in zip(nrns, cs):
        plt.eventplot(nrn.spikes['times'], lineoffsets=0, linelengths=0.5, colors=c)
    plt.xlabel('Time(ms)')
    plt.ylabel('Relative amp')
    plt.tight_layout()

try:
    plt.close(fig_spike_plasticity)
except:
    ...
fig_spike_plasticity, axes = plt.subplots(1,2,figsize=(10,5))
widgets.interact(update_spike_plasticity, delay_s=(1, 10, 1), delay_t=(1, 10, 1), Itype=Itypes);

## To-Do

Explore how the final synaptic weight change depends on the relative timing of spikes between the pre- and post-synaptic neurons.  
Systematically vary the delay between their inputs to measure weight change as a function of spike timing.

## Think

How does synaptic weight change differ between regular and irregular (e.g., Poisson) firing patterns?  
What might explain the differences in the resulting plasticity?
